<a href="https://colab.research.google.com/github/espada105/BitcoinPricePrediction/blob/main/datapreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=afcad0a7a1df88e8a06398e1328996b190e5ffc6ecc7f361641a74bfde86045c
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import ta
import requests
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
# 1. 데이터 수집
def collect_data():
    print("OHLCV 데이터 수집 중...")
    # 비트코인 OHLCV 데이터 수집 (최근 5년)
    btc_ohlcv = yf.download("BTC-USD", period="5y", interval="1d")

    # 디버깅을 위한 데이터프레임 구조 출력
    print("OHLCV 데이터 구조:")
    print(f"인덱스 유형: {type(btc_ohlcv.index)}")
    print(f"컬럼 유형: {type(btc_ohlcv.columns)}")
    print(f"컬럼 레벨 수: {btc_ohlcv.columns.nlevels}")

    # MultiIndex 문제 해결 방법 1: 컬럼 이름을 간단히 변경
    if btc_ohlcv.columns.nlevels > 1:
        # 멀티레벨 컬럼을 단일 레벨로 바꾸되, 첫 번째 레벨만 사용
        btc_ohlcv.columns = [col[0] for col in btc_ohlcv.columns]

    # 인덱스를 명시적으로 날짜 열로 변환
    btc_ohlcv = btc_ohlcv.reset_index()

    print("공포/탐욕 지수 수집 중...")
    # 공포/탐욕 지수 데이터 수집
    try:
        fg_url = "https://api.alternative.me/fng/?limit=2000&format=json"
        # 최대 2000일치 데이터 요청, 하루에 한 번(미국 동부 시간) 기준으로 업데이트

        fg_response = requests.get(fg_url)
        fg_data = pd.DataFrame(fg_response.json()['data'])
        # 명시적으로 timestamp를 정수로 변환 후 날짜로 변환
        fg_data['timestamp'] = fg_data['timestamp'].astype(int)
        fg_data['Date'] = pd.to_datetime(fg_data['timestamp'], unit='s')
        fg_data['fear_greed_index'] = fg_data['value'].astype(float)
        fg_data = fg_data[['Date', 'fear_greed_index']]
    except Exception as e:
        print(f"공포/탐욕 지수 수집 오류: {e}")
        # 오류 발생 시 빈 데이터프레임 생성
        fg_data = pd.DataFrame(columns=['Date', 'fear_greed_index'])

    print("기본 데이터 프레임 구성 중...")
    # 먼저 날짜를 기준으로 병합
    if not fg_data.empty:
        # 두 데이터프레임 모두 'Date' 열이 있는지 확인
        print(f"OHLCV 데이터 컬럼: {btc_ohlcv.columns.tolist()}")
        print(f"공포/탐욕 데이터 컬럼: {fg_data.columns.tolist()}")

        # 명시적으로 컬럼 타입 맞추기
        btc_ohlcv['Date'] = pd.to_datetime(btc_ohlcv['Date'])
        fg_data['Date'] = pd.to_datetime(fg_data['Date'])

        # 날짜 기준으로 데이터 병합
        data = pd.merge(btc_ohlcv, fg_data, on='Date', how='left')
    else:
        data = btc_ohlcv.copy()

    print("추가 데이터 생성 중...")
    # 랜덤 데이터 직접 추가
    data_len = len(data)

    # 추가 데이터 열 생성 - 컬럼 이름 변경 반영
    data['market_cap'] = data['Close'] * np.random.uniform(18000000, 19500000, size=data_len)
    data['bitcoin_search'] = np.random.randint(20, 100, size=data_len)
    data['transactions_per_day'] = np.random.randint(200000, 400000, size=data_len)
    data['hash_rate'] = np.random.randint(100, 300, size=data_len)
    data['difficulty'] = np.random.randint(10, 30, size=data_len)

    # 최종적으로 날짜를 인덱스로 설정
    data.set_index('Date', inplace=True)

    print(f"수집된 데이터: {len(data)}행 x {len(data.columns)}열")
    return data

# 2. 기술적 지표 생성
def create_technical_indicators(df):
    print("기술적 지표 계산 중...")

    # 'ta' 라이브러리 사용 (pandas_ta 대신)
    import ta

    # 기본 기술적 지표 계산
    # RSI
    df['rsi_14'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()

    # 이동평균선
    df['sma_20'] = ta.trend.SMAIndicator(df['Close'], window=20).sma_indicator()
    df['sma_50'] = ta.trend.SMAIndicator(df['Close'], window=50).sma_indicator()
    df['sma_200'] = ta.trend.SMAIndicator(df['Close'], window=200).sma_indicator()

    # MACD
    macd_indicator = ta.trend.MACD(df['Close'], window_slow=26, window_fast=12, window_sign=9)
    df['MACD_12_26_9'] = macd_indicator.macd()
    df['MACDs_12_26_9'] = macd_indicator.macd_signal()
    df['MACDh_12_26_9'] = macd_indicator.macd_diff()

    # 볼린저 밴드
    bollinger = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2)
    df['BBM_20_2.0'] = bollinger.bollinger_mavg()
    df['BBU_20_2.0'] = bollinger.bollinger_hband()
    df['BBL_20_2.0'] = bollinger.bollinger_lband()

    # 가격 모멘텀
    df['return_1d'] = df['Close'].pct_change(1)
    df['return_7d'] = df['Close'].pct_change(7)
    df['return_30d'] = df['Close'].pct_change(30)

    # 변동성 지표
    df['volatility_30d'] = df['return_1d'].rolling(window=30).std()

    # 추가 지표
    df['ema_9'] = ta.trend.EMAIndicator(df['Close'], window=9).ema_indicator()

    # 거래량 기반 지표
    df['volume_sma_20'] = ta.trend.SMAIndicator(df['Volume'], window=20).sma_indicator()
    df['volume_ratio'] = df['Volume'] / df['volume_sma_20']

    # 추가 지표
    # 스토캐스틱 오실레이터
    stoch = ta.momentum.StochasticOscillator(df['High'], df['Low'], df['Close'], window=14, smooth_window=3)
    df['stoch_k'] = stoch.stoch()
    df['stoch_d'] = stoch.stoch_signal()

    # ATR (Average True Range) - 변동성 지표
    df['atr'] = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Close'], window=14).average_true_range()

    print("기술적 지표 계산 완료")
    return df

# 3. 누락값 처리
def handle_missing_values(df):
    # 누락값 확인
    print("누락값 수:", df.isna().sum())

    # 기술적 지표로 인한 초기 누락값 제거
    df = df.dropna(subset=['sma_200', 'volatility_30d'])

    # 나머지 누락값은 전진 채우기
    df = df.ffill()

    # 여전히 남은 누락값이 있다면 후진 채우기
    df = df.bfill()

    return df

# 4. 이상치 처리
def handle_outliers(df):
    # 수치형 변수에 대해 IQR 방식으로 이상치 탐지 및 처리
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        if col in ['Open', 'High', 'Low', 'Close', 'Volume']:  # 가격/거래량은 이상치 처리하지 않음
            continue

        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # 이상치를 경계값으로 대체
        df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
        df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])

    return df

# 5. 특성 엔지니어링
def feature_engineering(df):
    # 시간 기반 특성
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['quarter'] = df.index.quarter
    df['year'] = df.index.year

    # 가격과 지표 사이의 비율
    df['price_to_sma20'] = df['Close'] / df['sma_20']
    df['price_to_sma50'] = df['Close'] / df['sma_50']
    df['price_to_sma200'] = df['Close'] / df['sma_200']

    # 지연 변수 생성 (1일, 7일, 14일, 30일 전 종가)
    for lag in [1, 7, 14, 30]:
        df[f'close_lag_{lag}'] = df['Close'].shift(lag)

    # 거래량과 가격 상호작용
    df['volume_price_change'] = df['Volume'] * df['return_1d'].abs()

    return df

# 6. 스케일링
def scale_features(df, target_col='Close'):
    print("데이터 스케일링 중...")
    # 타겟 변수 분리
    y = df[target_col].values

    # 예측에 사용할 특성 후보 목록
    potential_features = [
        'Open', 'High', 'Low', 'Volume',
        'rsi_14', 'sma_20', 'sma_50', 'sma_200',
        'MACDh_12_26_9', 'BBL_20_2.0', 'BBU_20_2.0',
        'return_1d', 'return_7d', 'volatility_30d',
        'fear_greed_index', 'transactions_per_day', 'hash_rate',
        'day_of_week', 'month', 'price_to_sma20', 'price_to_sma50',
        'close_lag_1', 'close_lag_7', 'volume_price_change',
        'bitcoin_search', 'is_weekend', 'price_up', 'uptrend',
        'stoch_k', 'stoch_d', 'atr'
    ]

    # 실제로 존재하는 컬럼만 선택
    available_features = [col for col in potential_features if col in df.columns]
    print(f"사용 가능한 특성: {len(available_features)}개")
    print("사용 특성 목록:", available_features)

    X = df[available_features].values

    # 스케일링 (MinMaxScaler 사용)
    scaler_X = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler_X.fit_transform(X)

    # 타겟 변수도 스케일링
    scaler_y = MinMaxScaler(feature_range=(0, 1))
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

    return X_scaled, y_scaled, scaler_X, scaler_y, available_features

# 7. 시계열 데이터 분할 (학습/검증/테스트)
def split_time_series_data(X, y, test_size=0.2, val_size=0.2):
    n_samples = len(X)

    # 테스트셋 분리
    test_split_idx = int(n_samples * (1 - test_size))
    X_train_val, X_test = X[:test_split_idx], X[test_split_idx:]
    y_train_val, y_test = y[:test_split_idx], y[test_split_idx:]

    # 검증셋 분리
    val_split_idx = int(len(X_train_val) * (1 - val_size))
    X_train, X_val = X_train_val[:val_split_idx], X_train_val[val_split_idx:]
    y_train, y_val = y_train_val[:val_split_idx], y_train_val[val_split_idx:]

    print(f"훈련 데이터: {len(X_train)} 샘플")
    print(f"검증 데이터: {len(X_val)} 샘플")
    print(f"테스트 데이터: {len(X_test)} 샘플")

    return X_train, X_val, X_test, y_train, y_val, y_test

# 8. 시계열 예측 데이터 준비 (시퀀스 생성)
def create_sequences(X, y, seq_length=30):
    """
    시퀀스 기반 모델(LSTM 등)을 위한 데이터 준비
    """
    X_seq, y_seq = [], []

    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])

    return np.array(X_seq), np.array(y_seq)

# 전체 파이프라인 실행
def main():
    # 1. 데이터 수집
    print("데이터 수집 중...")
    df = collect_data()

    # 2. 기술적 지표 생성
    print("기술적 지표 생성 중...")
    df = create_technical_indicators(df)

    # 3. 누락값 처리
    print("누락값 처리 중...")
    df = handle_missing_values(df)

    # 4. 이상치 처리
    print("이상치 처리 중...")
    df = handle_outliers(df)

    # 5. 특성 엔지니어링
    print("특성 엔지니어링 중...")
    df = feature_engineering(df)

    # 6. 스케일링
    print("데이터 스케일링 중...")
    X_scaled, y_scaled, scaler_X, scaler_y, feature_cols = scale_features(df)

    # 7. 데이터 분할
    print("데이터 분할 중...")
    X_train, X_val, X_test, y_train, y_val, y_test = split_time_series_data(X_scaled, y_scaled)

    # 8. 시퀀스 생성 (LSTM 등을 위한 추가 준비)
    print("시퀀스 데이터 생성 중...")
    seq_length = 30  # 30일간의 데이터로 다음 날 예측
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, seq_length)
    X_val_seq, y_val_seq = create_sequences(X_val, y_val, seq_length)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, seq_length)

    print("데이터 준비 완료!")
    print(f"시퀀스 훈련 데이터 형태: {X_train_seq.shape}")
    print(f"시퀀스 검증 데이터 형태: {X_val_seq.shape}")
    print(f"시퀀스 테스트 데이터 형태: {X_test_seq.shape}")

    return df, X_train_seq, X_val_seq, X_test_seq, y_train_seq, y_val_seq, y_test_seq, scaler_y, feature_cols

In [ ]:
main()

[*********************100%***********************]  1 of 1 completed

데이터 수집 중...
OHLCV 데이터 수집 중...
OHLCV 데이터 구조:
인덱스 유형: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
컬럼 유형: <class 'pandas.core.indexes.multi.MultiIndex'>
컬럼 레벨 수: 2
공포/탐욕 지수 수집 중...


기본 데이터 프레임 구성 중...
OHLCV 데이터 컬럼: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
공포/탐욕 데이터 컬럼: ['Date', 'fear_greed_index']
추가 데이터 생성 중...
수집된 데이터: 1827행 x 11열
기술적 지표 생성 중...
기술적 지표 계산 중...
기술적 지표 계산 완료
누락값 처리 중...
누락값 수: Close                     0
High                      0
Low                       0
Open                      0
Volume                    0
fear_greed_index          1
market_cap                0
bitcoin_search            0
transactions_per_day      0
hash_rate                 0
difficulty                0
rsi_14                   13
sma_20                   19
sma_50                   49
sma_200                 199
MACD_12_26_9             25
MACDs_12_26_9            33
MACDh_12_26_9            33
BBM_20_2.0               19
BBU_20_2.0               19
BBL_20_2.0               19
return_1d                 1
return_7d                 7
return_30d               30
volatility_30d           30
ema_9                     8
volume_sma_20            19
volume_ratio       

(                   Close          High           Low          Open  \
 Date                                                                 
 2020-10-14  11429.506836  11539.977539  11307.831055  11429.047852   
 2020-10-15  11495.349609  11569.914062  11303.603516  11426.602539   
 2020-10-16  11322.123047  11540.061523  11223.012695  11502.828125   
 2020-10-17  11358.101562  11386.261719  11285.345703  11322.123047   
 2020-10-18  11483.359375  11483.359375  11347.578125  11355.982422   
 ...                  ...           ...           ...           ...   
 2025-03-25  87471.703125  88542.398438  86346.078125  87512.820312   
 2025-03-26  86900.882812  88292.156250  85861.453125  87460.234375   
 2025-03-27  87177.101562  87786.726562  85837.937500  86896.257812   
 2025-03-28  84353.148438  87489.859375  83557.640625  87185.234375   
 2025-03-29  83889.343750  84543.101562  83658.312500  84370.781250   
 
                  Volume  fear_greed_index    market_cap  bitcoin_search  \

# 결과요약

## 실행 결과 요약

1. **데이터 수집**:
   - Bitcoin 5년치 OHLCV 데이터: 1,827일
   - 공포/탐욕 지수 데이터 성공적으로 병합
   - 생성된 추가 데이터 (거래량, 해시레이트 등)

2. **기술적 지표**:
   - RSI, 이동평균선, MACD, 볼린저밴드 등 성공적으로 계산
   - 스토캐스틱, ATR 등 추가 지표 생성

3. **데이터 전처리**:
   - 누락값 처리: 기술적 지표 초기값 등 처리 완료
   - 이상치 처리 완료
   - 특성 엔지니어링 완료

4. **데이터 준비**:
   - 사용 가능한 특성: 28개 (모든 필요 특성 포함)
   - 데이터 분할: 훈련(1,041일), 검증(261일), 테스트(326일)
   - 시퀀스 데이터 형태: (샘플 수, 30일, 28개 특성)

## 다음 단계 진행 예정 (모델 구축, 훈련 평가)

1. **모델 구축**:
   - LSTM, GRU 등의 순환 신경망 모델
   - XGBoost, Random Forest 등의 전통적 머신러닝 모델
   - Transformer 기반 모델

2. **모델 훈련 및 평가**:
   - 훈련 데이터로 모델 학습
   - 검증 데이터로 하이퍼파라미터 튜닝
   - 테스트 데이터로 최종 성능 평가

3. **백테스팅**:
   - 실제 트레이딩 전략에 모델 예측을 적용
   - 수익성 및 위험 지표 계산

> 30일간의 데이터를 사용해서 다음날 가격 예측에 적합함 그렇기 때문에 다른 모델이랑 비교해야될듯함